In [1]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import soundfile as sf
from scipy.fft import rfft,rfftfreq
import librosa
import librosa.display
import IPython.display as ipd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture

warnings.filterwarnings('ignore')

In [2]:
def mfcc_feature_extractor(audio,sampleRate):
    mfccsFeatures = librosa.feature.mfcc(y=audio, sr=sampleRate, n_mfcc=40)
    mfccsScaledFeatures = np.mean(mfccsFeatures.T,axis=0)
    
    return mfccsScaledFeatures

In [3]:
def contrast_feature_extractor(audio,sampleRate):
    stft = np.abs(librosa.stft(audio))
    contrast = librosa.feature.spectral_contrast(S=stft, sr=sampleRate) 
    contrastScaled = np.mean(contrast.T,axis=0)
    
    return contrastScaled

In [4]:
def tonnetz_feature_extractor(audio,sampleRate):
    tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(audio), sr=sampleRate)
    tonnetzScaled = np.mean(tonnetz.T,axis=0)
    
    return tonnetzScaled

In [6]:
def chroma_feature_extractor(audio,sampleRate):
    stft = np.abs(librosa.stft(audio))
    chroma = librosa.feature.chroma_stft(S=stft,sr=sampleRate)
    chromaScaled = np.mean(chroma.T,axis=0)
    
    return chromaScaled

In [7]:
def features_extractor(file):
    features=[]
    audio, sampleRate = librosa.load(file, res_type='kaiser_fast') 
    mfcc=mfcc_feature_extractor(audio,sampleRate)
    contrast = contrast_feature_extractor(audio,sampleRate)
    tonnetz = tonnetz_feature_extractor(audio,sampleRate)
    chroma = chroma_feature_extractor(audio,sampleRate)

    features.append([mfcc,contrast,tonnetz,chroma])
    features[0] = np.concatenate((features[0][0],features[0][1],features[0][2],features[0][3]))
    return features

In [8]:
def add_features(extractedFeatures,dirPath,label):
    for file in os.listdir(dirPath):
        filePath = dirPath+'/'+file
        finalClassLabels=label
        audio, sampleRate = librosa.load(filePath, res_type='kaiser_fast') 
        mfcc=mfcc_feature_extractor(audio,sampleRate)
        contrast = contrast_feature_extractor(audio,sampleRate)
        tonnetz = tonnetz_feature_extractor(audio,sampleRate)
        chroma = chroma_feature_extractor(audio,sampleRate)
        extractedFeatures.append([mfcc,contrast,tonnetz,chroma,finalClassLabels])

In [ ]:
dict = {"in group":
            ["C:/Users/DELL/General/DSP Tasks/DSP_Task3/backend/files/Anwar",
            "C:/Users/DELL/General/DSP Tasks/DSP_Task3/backend/files/Aya",
            "C:/Users/DELL/General/DSP Tasks/DSP_Task3/backend/files/Ehab",
            "C:/Users/DELL/General/DSP Tasks/DSP_Task3/backend/files/Zeyad",
            "C:/Users/DELL/General/DSP Tasks/DSP_Task3/processing/anwar",
            "C:/Users/DELL/General/DSP Tasks/DSP_Task3/processing/aya",
            "C:/Users/DELL/General/DSP Tasks/DSP_Task3/processing/ehab",
            "C:/Users/DELL/General/DSP Tasks/DSP_Task3/processing/zeyad",
            "C:/Users/DELL/General/DSP Tasks/DSP_Task3/processing/words/Anwar",
            "C:/Users/DELL/General/DSP Tasks/DSP_Task3/processing/words/Aya",
            "C:/Users/DELL/General/DSP Tasks/DSP_Task3/processing/words/Ehab",
            "C:/Users/DELL/General/DSP Tasks/DSP_Task3/processing/words/Zeyad"],
                "out of group":
                    ["C:/Users/DELL/General/DSP Tasks/DSP_Task3/processing/others"]}

In [ ]:
extractedFeatures = []
for i in dict.keys():
    for x in dict[i]:
        add_features(extractedFeatures,x,i)

In [ ]:
data=pd.DataFrame(extractedFeatures,columns=['mfcc','contrast','tonnetz','chroma','class'])
print(data.shape)
data.head()

In [ ]:
features=data.iloc[:,0:-1]
target=data['class']
features.head()

In [ ]:
features = features.values.tolist()
for i in range(len(features)):
    features[i] = np.concatenate((features[i][0],features[i][1],features[i][2],features[i][3]))

In [ ]:
encoder = LabelEncoder()
target = encoder.fit_transform(target)

In [ ]:
xTrain, xTest, yTrain, yTest = train_test_split(features, target, test_size=0.25, random_state=0)

In [ ]:
classifier= SVC(kernel='linear',decision_function_shape="ovo") 
classifier.fit(xTrain,yTrain)

In [ ]:
preds = classifier.predict(xTest)

In [ ]:
def prediction(pred):
    if pred==0:
        print("in group")
    elif pred==1:
        print("out of group")


In [ ]:
print(classification_report(yTest,preds))

In [ ]:
testFeatures = features_extractor("C:/Users/DELL/General/DSP Tasks/DSP_Task3/processing/words/Aya/voice01100741.wav")
pred = classifier.predict(testFeatures)
prediction(pred)


In [ ]:
pickle_out = open("ingroup_model.pkl", "wb")
pickle.dump(classifier, pickle_out)
pickle_out.close()